In [13]:
import numpy as np
import open3d as o3d
import copy
import pickle
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
from matplotlib.colors import LinearSegmentedColormap
from shapely.geometry import Polygon

from OCC.Core.STEPControl import STEPControl_Reader
from OCC.Core.IFSelect import IFSelect_RetDone
from OCC.Core.TopAbs import TopAbs_WIRE, TopAbs_EDGE, TopAbs_VERTEX
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopoDS import topods
from OCC.Core.BRep import BRep_Tool
from OCC.Core.BRepAdaptor import BRepAdaptor_Curve
from OCC.Core.GeomAbs import GeomAbs_Circle, GeomAbs_Ellipse, GeomAbs_Line, GeomAbs_BSplineCurve, GeomAbs_Hyperbola, GeomAbs_Parabola
from OCC.Core.TopAbs import TopAbs_WIRE, TopAbs_EDGE, TopAbs_VERTEX
from OCC.Display.SimpleGui import init_display
from OCC.Core.BRepOffsetAPI import BRepOffsetAPI_MakeOffset
from OCC.Core.Quantity import Quantity_Color, Quantity_TOC_RGB
from OCC.Core.gp import gp_Pnt
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeVertex
from PyQt5 import QtWidgets
from OCC.Core.GCPnts import GCPnts_UniformDeflection
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepGProp import brepgprop
from scipy.spatial import ConvexHull

In [ ]:
# Read cad model data (STEP file)
width = 0.5

step_file_path = '/home/chris/Code/PointClouds/data/other_files/MortenPartSTEPVersion.STEP'

def load_step_file(file_path):
    step_reader = STEPControl_Reader()
    status = step_reader.ReadFile(file_path)
    if status == IFSelect_RetDone:
        step_reader.TransferRoots()
        shape = step_reader.OneShape()
        return shape
    else:
        raise Exception("Error: Cannot read STEP file.")

shape = load_step_file(step_file_path)

print('Step file has been loaded successfully.')

In [ ]:
def extract_top_edges(shape, deflection=0.1):
    edges = []
    max_z = -float('inf')
    edge_z_coordinates = []

    # First pass to collect the highest Z-coordinate of each edge
    exp_edge = TopExp_Explorer(shape, TopAbs_EDGE)
    while exp_edge.More():
        edge = topods.Edge(exp_edge.Current())
        edge_max_z = -float('inf')
        exp_vertex = TopExp_Explorer(edge, TopAbs_VERTEX)
        vertices = []
        while exp_vertex.More():
            vertex = topods.Vertex(exp_vertex.Current())
            point = BRep_Tool.Pnt(vertex)
            vertices.append(point)
            if point.Z() > edge_max_z:
                edge_max_z = point.Z()
            exp_vertex.Next()
        edge_z_coordinates.append((edge, edge_max_z, vertices))
        if edge_max_z > max_z:
            max_z = edge_max_z
        exp_edge.Next()

    #print(f"Total number of edges: {len(edge_z_coordinates)}")
    #print(f"Maximum Z-coordinate found: {max_z}")

    # Collect edges on the top plane
    top_edges = []
    point_cloud = []
    for edge, z, vertices in edge_z_coordinates:
        if all(abs(vertex.Z() - max_z) < 1e-3 for vertex in vertices):  # Ensure all vertices are at the top surface
            top_edges.append(edge)
            #print(f"Top edge added with highest vertex Z: {z}")

            # Sample points along the edge
            curve = BRepAdaptor_Curve(edge)
            u_min, u_max = curve.FirstParameter(), curve.LastParameter()

            # Calculate the length of the edge
            linear_props = GProp_GProps()
            brepgprop.LinearProperties(edge, linear_props)
            length = linear_props.Mass()

            # Calculate the number of samples based on the length and the desired point density
            num_samples = int(length / deflection)

            for i in range(num_samples):
                u = u_min + i * (u_max - u_min) / (num_samples - 1)
                pnt = curve.Value(u)
                point_cloud.append((pnt.X(), pnt.Y(), pnt.Z()))  # Store the coordinates as a tuple

    print(f"Total number of top view edges: {len(top_edges)}")
    print(f"Total number of points in the cad point cloud: {len(point_cloud)}")
    return top_edges, point_cloud

# Visualize the top edges
def visualize_top_edges(top_edges):
    display, start_display, add_menu, add_function_to_menu = init_display()

    # Assign the color red to each top edge
    red_color = Quantity_Color(1.0, 0.0, 0.0, Quantity_TOC_RGB)
    
    # Display each top edge with the red color
    for edge in top_edges:
        display.DisplayShape(edge, update=True, color=red_color)

    start_display()


# Main execution
print('Generating top view edges and sampling to create the associated point cloud...')
top_edges, cad_points = extract_top_edges(shape)
# if top_edges:
#     visualize_top_edges(top_edges)
# else:
#     print("Top edges not found.")

cad_pcd = o3d.geometry.PointCloud()
cad_pcd.points = o3d.utility.Vector3dVector(np.asarray(cad_points))
o3d.visualization.draw_geometries([cad_pcd])

print('Top view cad point cloud created.')

In [ ]:
# Apply clustering to the cad_pcd

eps = 0.50
min_points = 10

cad_shapes_labels = np.array(cad_pcd.cluster_dbscan(eps=eps, min_points=min_points, print_progress=True))

max_label = cad_shapes_labels.max()
print(f"Cad point cloud has {max_label + 1} clusters")
base_cmap = plt.get_cmap("tab20")
color_cycle = [base_cmap(i % 20) for i in range(max_label + 1)]
colors = np.array(color_cycle)[cad_shapes_labels]
colors[cad_shapes_labels<0] = 0
cad_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

cad_points_dictionary = {}
cad_centers_dictionary = {}

# Extract the points and calculate the centers for each cluster
for label in np.unique(cad_shapes_labels):
    if label != -1:  # Ignore noise
        cluster_points = np.asarray(cad_pcd.points)[cad_shapes_labels == label]
        cad_points_dictionary[label] = cluster_points
        cad_centers_dictionary[label] = np.mean(cluster_points, axis=0)

cad_centers_pcd = o3d.geometry.PointCloud()
cad_centers_points = list(cad_centers_dictionary.values())
cad_centers_pcd.points = o3d.utility.Vector3dVector(np.array(cad_centers_points))
cad_centers_pcd.paint_uniform_color([0, 1, 0])  # Color centers blue

o3d.visualization.draw_geometries([cad_pcd])

In [11]:
class ConcaveHullCluster:
    def __init__(self, cluster, k):
        cluster = np.unique(cluster, axis=0)
        self.cluster = np.delete(cluster, 2, 1)
        self.number_of_points = self.cluster.shape[0]
        self.section_cloud = False
        self.grid_size = 3
        if self.number_of_points >= 10000:
            self.section_cloud = True
            self.create_sections()
        self.direction_vector = np.array([-1, 0])
        self.grid_points = np.ones(self.cluster.shape[0], dtype=bool)
        self.hull_points_final = np.zeros(self.cluster.shape[0], dtype=bool)
        self.deactivated_point_indices = []
        self.k = k
        self.k_list = [8, 10, 12]
        self.hull_point_indices_global = []
        self.hull_point_indices_local = []
        self.it = 0
        self.breakloop_it = 40000
        self.collinearity = False
        self.current_section_index = None
        self.current_section_points = None
        self.current_section_points_global_indices = None
    
    def create_sections(self):
        #print('Creating Sections', flush=True)
        x_min, y_min = self.cluster.min(axis=0)
        x_max, y_max = self.cluster.max(axis=0)
        
        #print(f'x_max={x_max}, x_min={x_min}, y_max={y_max}, y_min={y_min}')
        #print(f'x_diff = {(x_max - x_min)}, y_diff = {(y_max - y_min)}')
        M = round((x_max - x_min) / (3 * self.grid_size))
        N = round((y_max - y_min) / (3 * self.grid_size))
        #print(f'M={M} and N={N}')
        if N == 0: 
            y_intervals = np.linspace(y_min, y_max, 2)
        else: 
            y_intervals = np.linspace(y_min, y_max, 2 * N + 1)

        if M == 0: 
            x_intervals = np.linspace(x_min, x_max, 2)
        else:
            x_intervals = np.linspace(x_min, x_max, 2 * M + 1)

        small_sections = []
        small_section_centers = []
        small_section_indices = []

        for i in range(max(2 * N, len(y_intervals) - 1)):
            for j in range(max(2 * M, len(x_intervals) - 1)):
                x_start, x_end = x_intervals[j], x_intervals[j + 1]
                y_start, y_end = y_intervals[i], y_intervals[i + 1]

                x_center = (x_start + x_end) / 2.0
                y_center = (y_start + y_end) / 2.0
                small_section_centers.append([x_center, y_center])
                
                if i == max(2 * N, len(y_intervals) - 1) - 1:
                    y_condition = (self.cluster[:, 1] >= y_start) & (self.cluster[:, 1] <= y_end)
                else:
                    y_condition = (self.cluster[:, 1] >= y_start) & (self.cluster[:, 1] < y_end)
                    
                if j == max(2 * M, len(x_intervals) - 1) - 1:
                    x_condition = (self.cluster[:, 0] >= x_start) & (self.cluster[:, 0] <= x_end)
                else:
                    x_condition = (self.cluster[:, 0] >= x_start) & (self.cluster[:, 0] < x_end)

                section_indices = np.where(x_condition & y_condition)[0]

                small_sections.append(self.cluster[section_indices])
                small_section_indices.append(section_indices)
        
        #print(f'Created {len(small_sections)} small sections.')

        self.section_centers = []
        self.section_indices_arrays = []
        self.section_points_arrays = []

        if N >= 1 and M >= 1:
            for i in range(min(2 * N - 1, len(y_intervals) - 2)):
                for j in range(min(2 * M - 1, len(x_intervals) - 2)):
                    x_start = x_intervals[j]
                    x_end = x_intervals[j + 2]
                    y_start = y_intervals[i]
                    y_end = y_intervals[i + 2]

                    x_center = (x_start + x_end) / 2.0
                    y_center = (y_start + y_end) / 2.0
                    self.section_centers.append([x_center, y_center])
                    
                    #print(f'Section {i*(2*M-1)+j+1} contains small sections {j+2*M*i}, {j+1+2*M*i}, {j+2*M*(i+1)} and {j+1+2*M*(i+1)}')

                    section_indices = np.concatenate([
                        small_section_indices[j + 2 * M * i],
                        small_section_indices[j + 1 + 2 * M * i],
                        small_section_indices[j + 2 * M * (i + 1)],
                        small_section_indices[j + 1 + 2 * M * (i + 1)]
                    ])
                    
                    self.section_indices_arrays.append(section_indices)
                    self.section_points_arrays.append(self.cluster[section_indices])

                    #print(f'Created section {len(self.section_centers)} with indices: {section_indices}')
                    #print(f'Section points set size: {len(self.section_points_sets[-1])}')

        elif N==0 and M>=1:
            for j in range(2*M-1):
                x_start = x_intervals[j]
                x_end = x_intervals[j+2]
                y_start = y_intervals[0]
                y_end = y_intervals[1]

                x_center = (x_start + x_end) / 2.0
                y_center = (y_start + y_end) / 2.0
                self.section_centers.append([x_center, y_center])

                section_indices = np.concatenate([
                    small_section_indices[j],
                    small_section_indices[j+1],
                ])

                self.section_indices_arrays.append(section_indices)
                self.section_points_arrays.append(self.cluster[section_indices])

                #print(f'Created section {len(self.section_centers)} with indices: {section_indices}')
                #print(f'Section points set size: {len(self.section_points_sets[-1])}')


        elif N>=1 and M==0:
            for i in range(2*N-1):
                x_start = x_intervals[0]
                x_end = x_intervals[1]
                y_start = y_intervals[i]
                y_end = y_intervals[i+2]

                x_center = (x_start + x_end) / 2.0
                y_center = (y_start + y_end) / 2.0
                self.section_centers.append([x_center, y_center])

                section_indices = np.concatenate([
                    small_section_indices[i],
                    small_section_indices[i+1],
                ])

                self.section_indices_arrays.append(section_indices)
                self.section_points_arrays.append(self.cluster[section_indices])

                #print(f'Created section {len(self.section_centers)} with indices: {section_indices}')
                #print(f'Section points set size: {len(self.section_points_sets[-1])}')

        elif N==0 and M==0:
            self.section_cloud = False

        #print(f'Total sections created: {len(self.section_centers)}', flush=True)


    def calculate_distances(self, hull_point, active_indices):
        if self.section_cloud:
            return np.sqrt(np.sum(np.square(self.current_section_points[active_indices] - hull_point), axis=1))
        else:
            return np.sqrt(np.sum(np.square(self.cluster[active_indices] - hull_point), axis=1))
    
    def find_closest_section(self, current_hull_point):
        distances = np.sqrt(np.sum(np.square(self.section_centers - current_hull_point), axis=1))
        return np.argsort(distances)[0]
    
    def get_knn(self, current_hull_point):
        if self.section_cloud:
            deactivated_mask = np.isin(self.current_section_points_global_indices, self.deactivated_point_indices)
            active_grid_points = np.logical_and(~deactivated_mask, self.grid_points)
            active_grid_points_indices_local = np.nonzero(active_grid_points)[0]

            distances = self.calculate_distances(current_hull_point, active_grid_points_indices_local)
            sorted_indices = np.argsort(distances)

            neighbor_indices = active_grid_points_indices_local[sorted_indices[:self.k]]
            neighbor_points = self.current_section_points[neighbor_indices]
            return neighbor_indices, neighbor_points
        else:
            inactive_indices = np.zeros(self.grid_points.shape, dtype=bool)
            inactive_indices[self.deactivated_point_indices] = True
            active_grid_points = np.logical_and(~inactive_indices, self.grid_points)
            active_grid_points_indices = np.nonzero(active_grid_points)[0]
            distances = self.calculate_distances(current_hull_point, active_grid_points_indices)
            sorted_indices = np.argsort(distances)
            neighbor_indices = active_grid_points_indices[sorted_indices[:self.k]]
            neighbor_points = self.cluster[neighbor_indices]
            return neighbor_indices, neighbor_points
    
    def find_lowest_point(self):
        lowest_y = np.amin(self.cluster[:,1])
        lowest_y_indices = np.where(self.cluster[:, 1] == lowest_y)[0]

        if lowest_y_indices.shape[0] == 1:
            lowest_point_index = lowest_y_indices[0]
            lowest_point = self.cluster[lowest_point_index]
        elif lowest_y_indices.shape[0] > 1:
            lowest_y_points = self.cluster[lowest_y_indices]
            lowest_x = np.amin(lowest_y_points[:, 0])
            lowest_point_index = np.where((self.cluster[:,0] == lowest_x) & (self.cluster[:,1] == lowest_y))[0][0]
            lowest_point = np.array([lowest_x, lowest_y])
        return lowest_point, lowest_point_index
    
    def get_points_inside_grid(self, current_hull_point):
        grid_size_2d = np.array([self.grid_size, self.grid_size])
        grid_min = current_hull_point - grid_size_2d / 2
        grid_max = current_hull_point + grid_size_2d / 2
        if self.section_cloud:
            #print(f'Current Section Points: {self.current_section_points}', flush=True)
            self.grid_points = np.all((self.current_section_points >= grid_min) & 
                                      (self.current_section_points <= grid_max), axis=1)
        else:
            self.grid_points = np.all((self.cluster >= grid_min) & (self.cluster <= grid_max), axis=1)

    def unit_vector(self, vector):
        return vector/np.linalg.norm(vector)
    
    def ccw(self,A,B,C):
        return (C[1]-A[1])*(B[0]-A[0]) > (B[1]-A[1])*(C[0]-A[0])

    def check_collinearity(self, A, B, C):
        area = A[0]*(B[1]-C[1]) + B[0]*(C[1]-A[1]) + C[0]*(A[1]-B[1])
        if area==0:
            return True
        return False

    def check_parallelism(self, vector_alpha, vector_beta):
        determinant = vector_alpha[0] * vector_beta[1] - vector_alpha[1] * vector_beta[0]
        return np.isclose(determinant, 0)

    def intersect_with_collinearity(self, A, B, C, D):
        if np.array_equal(A, C) or np.array_equal(A, D) or np.array_equal(B, C) or np.array_equal(B, D):
            return False
        if self.check_collinearity(A, C, D) or self.check_collinearity(B, C, D):
            return False
        return self.ccw(A, C, D) != self.ccw(B, C, D) and self.ccw(A, B, C) != self.ccw(A, B, D)
    
    def intersect_without_collinearity(self, A, B, C, D):
        if np.array_equal(A, C) or np.array_equal(A, D) or np.array_equal(B, C) or np.array_equal(B, D):
            return False
        return self.ccw(A, C, D) != self.ccw(B, C, D) and self.ccw(A, B, C) != self.ccw(A, B, D)
    
    def increase_k(self, current_hull_point, neighbors):
        for z in range(1, len(self.k_list)):
            self.k = self.k_list[z]
            print(f'Increasing k to {self.k}', flush=True)
            _, neighbors = self.get_knn(current_hull_point)
            chosen_neighbor = self.choose_neighbor(current_hull_point, neighbors)
            if chosen_neighbor is None:
                continue
            else:
                self.k = self.k_list[0]
                return chosen_neighbor

    def enable_collinearity(self, current_hull_point, neighbors):
        print('Enabling collinearity')
        self.collinearity = True
        for z in range(len(self.k_list)):
            self.k = self.k_list[z]
            print(f'Trying with k: {self.k}', flush=True)
            _, neighbors = self.get_knn(current_hull_point)
            chosen_neighbor = self.choose_neighbor(current_hull_point, neighbors)
            if chosen_neighbor is None:
                continue
            else:
                self.k = self.k_list[0]
                self.collinearity = False
                return chosen_neighbor
        self.collinearity = False

    def find_all_indices(self, target_list, query_value):
        return [i for i, x in enumerate(target_list) if x == query_value]
    
    def are_consecutive(self, index1, index2):
        return abs(index1 - index2) == 1

    def choose_neighbor(self, current_hull_point, neighbors):
        vectors = neighbors - current_hull_point
        dot_products = np.dot(vectors, self.direction_vector)
        cross_products = np.cross(self.direction_vector, vectors)
        norms = np.linalg.norm(vectors, axis=1)
        cos_angles = np.clip(dot_products / norms, -1.0, 1.0)
        angles = np.arccos(cos_angles)

        right_turn_indices = np.where(cross_products <= 0)[0]
        left_turn_indices = np.where(cross_products > 0)[0]

        if self.it > 2:
            # Separate the 180-degree turns
            threshold = np.deg2rad(179.0)  # Set a threshold for near 180-degree turns
            right_turn_angles = angles[right_turn_indices]
            left_turn_angles = angles[left_turn_indices]

            right_turn_180_indices = right_turn_indices[right_turn_angles >= threshold]
            right_turn_non_180_indices = right_turn_indices[right_turn_angles < threshold]
            left_turn_180_indices = left_turn_indices[left_turn_angles >= threshold]
            left_turn_non_180_indices = left_turn_indices[left_turn_angles < threshold]

            # Sort non-180-degree turns
            sorted_right_turn_non_180_indices = right_turn_non_180_indices[np.argsort(-right_turn_angles[right_turn_angles < threshold])]
            sorted_left_turn_non_180_indices = left_turn_non_180_indices[np.argsort(left_turn_angles[left_turn_angles < threshold])]

            # Combine all indices
            combined_indices = np.concatenate([sorted_right_turn_non_180_indices, sorted_left_turn_non_180_indices, right_turn_180_indices, left_turn_180_indices])
        else:
            right_turn_angles = angles[right_turn_indices]
            left_turn_angles = angles[left_turn_indices]
            sorted_right_turn_indices = right_turn_indices[np.argsort(-right_turn_angles)]
            sorted_left_turn_indices = left_turn_indices[np.argsort(left_turn_angles)]
            combined_indices = np.concatenate([sorted_right_turn_indices, sorted_left_turn_indices])
        
        if self.section_cloud:
            grid_hull_points_indices = [idx for idx in self.hull_point_indices_local if self.grid_points[idx]]
        else:
            grid_hull_points_indices = [idx for idx in self.hull_point_indices_global if self.grid_points[idx]]

        for index in combined_indices:
            chosen_neighbor = neighbors[index]
            C = current_hull_point
            D = chosen_neighbor
            intersects = False

            for i in range(1, len(grid_hull_points_indices)):
                A_index = grid_hull_points_indices[i - 1]
                B_index = grid_hull_points_indices[i]
                
                if self.section_cloud:
                    A_indices = self.find_all_indices(self.hull_point_indices_local, A_index)
                    B_indices = self.find_all_indices(self.hull_point_indices_local, B_index)
                else:
                    A_indices = self.find_all_indices(self.hull_point_indices_global, A_index)
                    B_indices = self.find_all_indices(self.hull_point_indices_global, B_index)

                is_consecutive = any(self.are_consecutive(A_idx, B_idx) for A_idx in A_indices for B_idx in B_indices)

                if is_consecutive:
                    if self.section_cloud:
                        A = self.current_section_points[A_index]
                        B = self.current_section_points[B_index]
                    else:
                        A = self.cluster[A_index]
                        B = self.cluster[B_index]

                    vector_alpha = B - A
                    vector_beta = D - C
                    if self.collinearity == False:
                        if self.intersect_without_collinearity(A, B, C, D) and not self.check_parallelism(vector_alpha, vector_beta):
                            intersects = True
                            break
                    else:
                        if self.intersect_with_collinearity(A, B, C, D) and not self.check_parallelism(vector_alpha, vector_beta):
                            intersects = True
                            break

            if not intersects:
                new_direction_vector = self.unit_vector(chosen_neighbor - current_hull_point)
                self.direction_vector = new_direction_vector
                return chosen_neighbor

        return None
    
    def indices_to_array(self):
        self.hull_points_final[self.hull_point_indices_global] = True
        return self.hull_points_final

    def get_ordered_hull_edges(self):
        ordered_points = self.cluster[self.hull_point_indices_global]  # Use global indices
        edges = [(ordered_points[i], ordered_points[i + 1]) for i in range(len(ordered_points) - 1)]
        edges.append((ordered_points[-1], ordered_points[0]))  # Close the loop
        return edges
    
    def compute_concave_hull(self):
        self.lowest_point, self.lowest_point_index = self.find_lowest_point()
        self.hull_point_indices_global.append(self.lowest_point_index)
        if self.section_cloud:
                self.current_section_index = self.find_closest_section(self.lowest_point)
                #print(f'Closest section to lowest point is Section {self.current_section_index + 1}', flush=True)
                self.current_section_points_global_indices = self.section_indices_arrays[self.current_section_index]
                self.current_section_points = self.section_points_arrays[self.current_section_index]
                self.hull_point_indices_local = [np.where(self.current_section_points_global_indices == idx)[0][0]
                                                  for idx in self.hull_point_indices_global if idx in 
                                                  self.current_section_points_global_indices]
        self.it += 1
        while True:
            self.k = self.k_list[0]
            #print(f'Iteration: {self.it}', end='\r', flush=True)
            if self.section_cloud and self.hull_point_indices_local:
                #print(f'number of local hull_points = {len(self.hull_point_indices_local)}', flush=True)
                #print(f'number of global hull_points = {len(self.hull_point_indices_global)}', flush=True)
                current_hull_point_global_index = self.hull_point_indices_global[-1]
                current_hull_point_local_index = self.hull_point_indices_local[-1]
                current_hull_point = self.current_section_points[current_hull_point_local_index]
            else:
                current_hull_point_global_index = self.hull_point_indices_global[-1]
                current_hull_point = self.cluster[current_hull_point_global_index]

            self.deactivated_point_indices.append(current_hull_point_global_index)

            if self.section_cloud:
                if self.it%3==0:
                    new_section_index = self.find_closest_section(current_hull_point)
                    if new_section_index != self.current_section_index:
                        #print('Changing sections!', flush=True)
                        self.current_section_index = new_section_index
                        self.current_section_points_global_indices = self.section_indices_arrays[self.current_section_index]
                        self.current_section_points = self.section_points_arrays[self.current_section_index]
                        self.hull_point_indices_local = [np.where(self.current_section_points_global_indices == idx)[0][0]
                                                for idx in self.hull_point_indices_global if idx in self.current_section_points_global_indices]
                        
        
            if self.it == self.breakloop_it:
                print('Time to stop this madness', flush=True)
                return self.indices_to_array()
            
            self.get_points_inside_grid(current_hull_point)
            _, neighbors = self.get_knn(current_hull_point)
            if self.section_cloud:
                _, neighbors = self.get_knn(current_hull_point)
            else:
                _, neighbors = self.get_knn(current_hull_point)
                #print(f'Neighborhood indices: {neighborhood_indices}', flush=True)
            chosen_neighbor = self.choose_neighbor(current_hull_point, neighbors)
            if chosen_neighbor is None:
                chosen_neighbor = self.increase_k(current_hull_point, neighbors)
            if chosen_neighbor is None:
                chosen_neighbor = self.enable_collinearity(current_hull_point, neighbors)
            if chosen_neighbor is None:
                print('Couldnt find neighbor or find lowest point, closing loop.', flush=True)
                return self.indices_to_array()
    
            if self.section_cloud:
                chosen_neighbor_index_local = np.where((self.current_section_points[:, 0] == chosen_neighbor[0]) & (self.current_section_points[:, 1] == chosen_neighbor[1]))[0][0]
                chosen_neighbor_index_global = self.current_section_points_global_indices[chosen_neighbor_index_local]
                self.hull_point_indices_local.append(chosen_neighbor_index_local)
            else:
                chosen_neighbor_index_global = np.where((self.cluster[:, 0] == chosen_neighbor[0]) & (self.cluster[:, 1] == chosen_neighbor[1]))[0][0]

            if chosen_neighbor_index_global == self.lowest_point_index:
                #print('Success!\n',flush=True)
                return self.indices_to_array()

            self.hull_point_indices_global.append(chosen_neighbor_index_global)
            if self.it >= int(2*self.k):
                del self.deactivated_point_indices[0]
            self.it +=1

In [ ]:
expanded_points_dictionary = {}
shrunk_points_dictionary = {}

for i, cluster_points in cad_points_dictionary.items():
    #print(f'Computing concave hull for cluster {i}...')
    concave_hull = ConcaveHullCluster(cluster_points, 2)
    hull = concave_hull.compute_concave_hull()
    hull_edges = concave_hull.get_ordered_hull_edges()
    polygon = Polygon([p[0] for p in hull_edges])
    # --- Expand/Shrink Polygon ---
    expanded_polygon = polygon.buffer(0.3)
    shrunk_polygon = polygon.buffer(-0.3)

    expanded_points = np.array(expanded_polygon.exterior.coords)
    shrunk_points = np.array(shrunk_polygon.exterior.coords)

    expanded_points_dictionary[i] = expanded_points
    shrunk_points_dictionary[i] = shrunk_points

    print(f'Expanded points shape: {expanded_points.shape}')
    print(f'Shrunk points shape: {shrunk_points.shape}')
    # # --- Visualization ---
    # plt.figure(figsize=(6, 6))

    # # Plot original concave hull
    # x, y = polygon.exterior.xy
    # plt.plot(x, y, 'b-', label="Original CAD Shape")

    # # Plot expanded polygon
    # x, y = expanded_polygon.exterior.xy
    # plt.plot(x, y, 'g-', label="Expanded (+0.3mm)")

    # # Plot shrunk polygon
    # x, y = shrunk_polygon.exterior.xy
    # plt.plot(x, y, 'r-', label="Shrunk (-0.3mm)")

    # plt.legend()
    # plt.axis('equal')
    # plt.title(f"Shape {i+1}/79")
    # plt.show()

Success!

Expanded points shape: (1812, 2)
Shrunk points shape: (981, 2)
Success!

Expanded points shape: (1641, 2)
Shrunk points shape: (821, 2)
Success!

Expanded points shape: (1161, 2)
Shrunk points shape: (581, 2)
Success!

Expanded points shape: (625, 2)
Shrunk points shape: (313, 2)
Success!

Expanded points shape: (625, 2)
Shrunk points shape: (313, 2)
Success!

Expanded points shape: (625, 2)
Shrunk points shape: (313, 2)
Success!

Expanded points shape: (625, 2)
Shrunk points shape: (313, 2)
Success!

Expanded points shape: (625, 2)
Shrunk points shape: (313, 2)
Success!

Expanded points shape: (625, 2)
Shrunk points shape: (313, 2)
Success!

Expanded points shape: (625, 2)
Shrunk points shape: (313, 2)
Success!

Expanded points shape: (625, 2)
Shrunk points shape: (313, 2)
Success!

Expanded points shape: (625, 2)
Shrunk points shape: (313, 2)
Success!

Expanded points shape: (625, 2)
Shrunk points shape: (313, 2)
Success!

Expanded points shape: (625, 2)
Shrunk points shape

In [36]:
import numpy as np
import open3d as o3d
from shapely.geometry import Polygon, LineString
from shapely.ops import unary_union

# Function to extract detailed points from a polygon, ensuring complete coverage
def extract_detailed_polygon_points(polygon, resolution=0.1):
    """Interpolates points along the polygon edges to prevent missing details."""
    if not polygon.is_valid:
        return np.array([])  # Return empty if the polygon is invalid
    
    detailed_points = []
    
    for segment in polygon.exterior.coords[:-1]:  # Avoid duplicate closing point
        detailed_points.append(segment)  

    # Generate interpolated points for each edge
    interpolated_points = []
    for i in range(len(detailed_points) - 1):
        start, end = np.array(detailed_points[i]), np.array(detailed_points[i + 1])
        dist = np.linalg.norm(end - start)
        num_samples = max(6, int(dist / resolution))  # Ensures at least two points per segment
        sampled_points = np.linspace(start, end, num_samples)
        interpolated_points.append(sampled_points)

    return np.vstack(interpolated_points)

# Initialize dictionaries
expanded_points_dictionary = {}
shrunk_points_dictionary = {}

for i, cluster_points in cad_points_dictionary.items():
    concave_hull = ConcaveHullCluster(cluster_points, 2)
    hull = concave_hull.compute_concave_hull()
    hull_edges = concave_hull.get_ordered_hull_edges()
    polygon = Polygon([p[0] for p in hull_edges])

    if not polygon.is_valid:
        print(f"Skipping shape {i}: Invalid Polygon")
        continue

    # Expand/Shrink Polygon
    expanded_polygon = polygon.buffer(0.3) if polygon.buffer(0.3).is_valid else None
    shrunk_polygon = polygon.buffer(-0.3) if polygon.buffer(-0.3).is_valid else None

    if expanded_polygon:
        expanded_points = extract_detailed_polygon_points(expanded_polygon, resolution=0.05)
        expanded_points_dictionary[i] = expanded_points
        print(f"Expanded points shape {i}: {expanded_points.shape}")

    if shrunk_polygon:
        shrunk_points = extract_detailed_polygon_points(shrunk_polygon, resolution=0.05)
        shrunk_points_dictionary[i] = shrunk_points
        print(f"Shrunk points shape {i}: {shrunk_points.shape}")


Success!

Expanded points shape 0: (10905, 2)
Shrunk points shape 0: (5920, 2)
Success!

Expanded points shape 1: (9834, 2)
Shrunk points shape 1: (4914, 2)
Success!

Expanded points shape 2: (6954, 2)
Shrunk points shape 2: (3474, 2)
Success!

Expanded points shape 3: (3738, 2)
Shrunk points shape 3: (1866, 2)
Success!

Expanded points shape 4: (3738, 2)
Shrunk points shape 4: (1866, 2)
Success!

Expanded points shape 5: (3738, 2)
Shrunk points shape 5: (1866, 2)
Success!

Expanded points shape 6: (3738, 2)
Shrunk points shape 6: (1866, 2)
Success!

Expanded points shape 7: (3738, 2)
Shrunk points shape 7: (1866, 2)
Success!

Expanded points shape 8: (3738, 2)
Shrunk points shape 8: (1866, 2)
Success!

Expanded points shape 9: (3738, 2)
Shrunk points shape 9: (1866, 2)
Success!

Expanded points shape 10: (3738, 2)
Shrunk points shape 10: (1866, 2)
Success!

Expanded points shape 11: (3738, 2)
Shrunk points shape 11: (1866, 2)
Success!

Expanded points shape 12: (3738, 2)
Shrunk points

In [40]:
# Convert to Open3D PointCloud
expanded_pcd = o3d.geometry.PointCloud()
if expanded_points_dictionary:
    expanded_pcd_points = np.vstack(list(expanded_points_dictionary.values()))
    expanded_pcd.points = o3d.utility.Vector3dVector(np.c_[expanded_pcd_points, np.zeros(len(expanded_pcd_points))])
    expanded_pcd.paint_uniform_color([0, 1, 0]) 

shrunk_pcd = o3d.geometry.PointCloud()
if shrunk_points_dictionary:
    shrunk_pcd_points = np.vstack(list(shrunk_points_dictionary.values()))
    shrunk_pcd.points = o3d.utility.Vector3dVector(np.c_[shrunk_pcd_points, np.zeros(len(shrunk_pcd_points))])
    shrunk_pcd.paint_uniform_color([1, 0, 0])

cad_points = np.asarray(cad_pcd.points)
cad_points[:, 2] = 0
cad_pcd.points = o3d.utility.Vector3dVector(cad_points)
cad_pcd.paint_uniform_color([0, 0, 0])

o3d.visualization.draw_geometries([cad_pcd, expanded_pcd, shrunk_pcd])